In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import Sequential
from keras import applications
from keras import optimizers
from keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Activation
from keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from scipy import misc, ndimage
%matplotlib inline

In [ ]:
train_path = 'images/train'
valid_path = 'images/valid'
test_path = 'images/test'

In [ ]:
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if(ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

In [ ]:
#Create Custom Image Data Generator
dataGen = ImageDataGenerator(    
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=False)

In [ ]:
#create train/valid/test directories
train_batches = dataGen.flow_from_directory(train_path, target_size=(224, 224), color_mode='rgb', classes=['excited', 'happy', 'sad'], class_mode='categorical')
valid_batches = dataGen.flow_from_directory(valid_path, target_size=(224, 224), color_mode='rgb', classes=['excited', 'happy', 'sad'], class_mode='categorical')
test_batches = dataGen.flow_from_directory(test_path, target_size=(224, 224), color_mode='rgb', classes=['excited', 'happy', 'sad'], class_mode='categorical')

In [ ]:
#Create Facial Response Model
model = Sequential()

#Convolution layers in segments 5x5 with 32 pattern recognition
#get max value from 2x2 window to prevent overfitting
model.add(Conv2D(32, (5, 5), padding='same', activation='relu', input_shape=(224, 224, 3)))
model.add(Conv2D(32, (5, 5), padding='same', activation='relu'))
model.add(Conv2D(32, (5, 5), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

#Convolution layers in segments 3x3 with 64 pattern recognition
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

#Convolution layers in segments 3x3 with 128 pattern recognition
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

#Create a 1D feature vector to create classifier
model.add(Flatten())

model.add(Dense(128, activation='relu'))
#Every training cycle leaves out 50% of the neurons for generalization purposes
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [ ]:
#Compile model using 
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [ ]:
#fit model using optimizer ADAM
model.fit_generator(train_batches
             ,steps_per_epoch= 1
             ,validation_data = valid_batches
             ,validation_steps= 1
             ,epochs=5
             ,verbose=2)

In [ ]:
model.predict_generator(test_batches,
                        steps=1,
                        verbose=0)

In [ ]:
test_img, test_labels = next(test_batches)
test_labels
#test_labels = test_labels[:,0]